In [ ]:
import cloudvolume
import pandas as pd
from imageryclient import ImageryClient
import imageryclient as ic
import matplotlib.pyplot as plt
import numpy as np
import scipy
from PIL import Image
import fastremap
import os

In [ ]:
# syn_df = pd.read_csv('../data/221206_pni_synapses_v185.csv')

In [ ]:
mito = pd.read_csv('../data/211019_mitochondria_info.csv')

In [ ]:
img_source = "precomputed://https://storage.googleapis.com/microns_public_datasets/pinky100_v0/son_of_alignment_v15_rechunked"
seg_source = "precomputed://https://storage.googleapis.com/microns_public_datasets/pinky100_v185/seg"
mito_source = "precomputed://https://seungdata.princeton.edu/sseung-archive/pinky100-mito/seg_191220"

In [ ]:
img_cv = cloudvolume.CloudVolume(img_source, use_https=True)

In [ ]:
img_cv.bounds

In [ ]:
# set desired image size

image_size = 1000
half_image_size = image_size/2

In [ ]:
# enter cellid of interest

cellid = 648518346349527319

# some interesting features including top and bottom positions
# 648518346349538787 [80509, 46999, 812] [80509, 46999, 868] #ER near a dendritic mito that is continuous into a dendritic spine
# 648518346349538089 2532746 [87729, 65165, 980] [87729, 65165, 1008] # astro mito with electron bright inclusion
# 648518346349538089 2646319 [88720, 64452, 1003] [88720, 64452, 1027] # astro mito with electron bright inclusion
# 648518346349538089 2528399 [87458, 63475, 600] [87458, 63475, 624] # astro mito with electron bright inclusion
# 648518346349538089 2644843 [88939, 62480, 1125] [88939, 62480, 1156] # astro mito inclusion 
# 648518346341354380 2643657 [87741, 62385, 2058] [87741, 62385, 2086] # astro mito with electron bright inclusion
# 648518346341354380 2643929 [87968, 61968, 2012] [87968, 61968, 2036] # astro mito with inclusion and continuity between two different mito ids
# 648518346341354380 3425723 [102919, 65891, 1934] [102919, 65891, 1958] # astro mito with inclusion and nearby large dendrite
# 648518346349527319 980814 [59564, 44866, 1425] [59564, 44866, 1454] # astro with mito inclusion

In [ ]:
# enter mitoid of interest for file/folder naming purposes

mitoid = 980814

In [ ]:
# "bottom" of desired volume (e.g., copy and paste from Neuroglancer)
position = [59564, 44866, 1425]

# "top" of desired volume
top_position = [59564, 44866, 1455]

In [ ]:
zslices = top_position[2] - position[2]
zslices

In [ ]:
# set up directories

imagestacks_dir = 'image_stacks/'+str(cellid)+'/'+str(mitoid)+'/' 
imagemontages_dir = 'image_montages/'+str(cellid)+'/'+str(mitoid)+'/'
single_images_dir = 'single_images/'+str(cellid)+'/'+str(mitoid)+'/'

In [ ]:
# create directory folders

os.makedirs(imagestacks_dir, exist_ok=True)
os.makedirs(imagemontages_dir, exist_ok=True)
os.makedirs(single_images_dir, exist_ok=True)

In [ ]:
print(imagestacks_dir)
print(imagemontages_dir)
print(single_images_dir)

In [ ]:
position[0]

In [ ]:
position_x = position[0]
position_y = position[1]
position_z = position[2]

position_x_min = position_x - half_image_size
position_x_max = position_x + half_image_size
position_y_min = position_y - half_image_size
position_y_max = position_y + half_image_size
position_z_max = position_z + 1

In [ ]:
img = img_cv[position_x_min:position_x_max, position_y_min:position_y_max, position_z:position_z_max]

In [ ]:
# img

In [ ]:
# show image from bottom position
# from: https://stackoverflow.com/questions/31401812/matplotlib-rotate-image-file-by-x-degrees
tr = scipy.ndimage.rotate(img, 90)
plt.imshow(np.squeeze(tr), cmap=plt.cm.gray, origin='lower')
#plt.gca().invert_yaxis()

# Add cell segmentation color for cellid of interest

In [ ]:
img_client = ic.ImageryClient(image_source=img_source, segmentation_source=seg_source)

In [ ]:
ctr = position
img_width = image_size
image = img_client.image_cutout(ctr, bbox_size=(img_width, img_width))

In [ ]:
image

In [ ]:
bounds = ic.bounds_from_center(ctr, width=img_width, height=img_width, depth=1)

In [ ]:
image = img_client.image_cutout(bounds)

# Use PIL to visualize
Image.fromarray(image.T)

In [ ]:
seg = img_client.segmentation_cutout(bounds)

# Image.fromarray( (seg.T / np.max(seg) * 255).astype('uint8') )
Image.fromarray( (seg.T).astype('uint8') )

In [ ]:
root_ids = [cellid]
seg = img_client.segmentation_cutout(bounds, root_ids=root_ids)
Image.fromarray( (seg.T / np.max(seg) * 255).astype('uint8') )

In [ ]:
split_seg = img_client.split_segmentation_cutout(bounds, root_ids=root_ids)

Image.fromarray((split_seg[ root_ids[0] ].T * 255).astype('uint8'))

In [ ]:
image, segs = img_client.image_and_segmentation_cutout(bounds,
                                                       split_segmentations=True,
                                                       root_ids=root_ids)

In [ ]:
ic.composite_overlay(segs, imagery=image)

In [ ]:
position

In [ ]:
# this is a repeat of the above code condensed into a single notebook cell

ctr = [position[0], position[1], position[2]+0] 
img_width = image_size
image = img_client.image_cutout(ctr, bbox_size=(img_width, img_width))
bounds = ic.bounds_from_center(ctr, width=img_width, height=img_width, depth=1)
image = img_client.image_cutout(bounds)
seg = img_client.segmentation_cutout(bounds)
root_ids = [cellid]
seg = img_client.segmentation_cutout(bounds, root_ids=root_ids)
split_seg = img_client.split_segmentation_cutout(bounds, root_ids=root_ids)
image, segs = img_client.image_and_segmentation_cutout(bounds, split_segmentations=True, root_ids=root_ids)
ic.composite_overlay(segs, imagery=image)

In [ ]:
zslices

In [ ]:
position

In [ ]:
top_position

In [ ]:
# ax

In [ ]:
# create a dictionary to store a segmentation image of each slice
# this will be more time-consuming as the slices and/or image size increases

ic_grid_dict = {}
for i in range(zslices):
    ctr = [position[0], position[1], position[2]+i] 
    img_width = image_size
    image = img_client.image_cutout(ctr, bbox_size=(img_width, img_width))
    bounds = ic.bounds_from_center(ctr, width=img_width, height=img_width, depth=1)
    image = img_client.image_cutout(bounds)
    seg = img_client.segmentation_cutout(bounds)
    root_ids = [cellid]
    seg = img_client.segmentation_cutout(bounds, root_ids=root_ids)
    split_seg = img_client.split_segmentation_cutout(bounds, root_ids=root_ids)
    image, segs = img_client.image_and_segmentation_cutout(bounds, split_segmentations=True, root_ids=root_ids)
    ic_grid_dict['im_' + str(i)] = ic.composite_overlay(segs, imagery=image)
    
locals().update(ic_grid_dict)

In [ ]:
# ic_grid_dict

In [ ]:
zslices

In [ ]:
# calculations to help you decide on a montage layout (e.g., number of rows and columns)

print(f"7 columns:", zslices/7)
print(f"5 columns:", zslices/5)
print(f"4 columns:", zslices/4)
print(f"3 columns:", zslices/3)
print(f"2 columns:", zslices/2)

In [ ]:
# manually enter the desired number of rows by columns for montage layout

nrows = 10
ncols = 3

## Save a montage image grid of all slices

In [ ]:
# change desired DPI; 300 dpi will result in very large montage file sizes
# uncomment fig lines to generate and save the montage image

dpi = 100
rows = nrows
cols = ncols
width = 30
height = (width*((rows)/(cols)))

fig, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(width, height))
plt.subplots_adjust(hspace=0)
k=0
for i in range(rows):
    for j in range(cols):
        plt.setp(ax, xticks=[], yticks=[])
        ax[i,j].imshow(ic_grid_dict['im_' + str(k)])
        k=k+1

plt.savefig(os.path.join(imagemontages_dir, 'cid_'+str(cellid)+'_mid_'+str(mitoid)+'_zmontage_'+str(dpi)+'dpi.png'), dpi=dpi)
plt.show()

In [ ]:
print(os.path.join(imagemontages_dir, 'cid_'+str(cellid)+'_mid_'+str(mitoid)+'_zmontage_'+str(dpi)+'dpi.png'))

## Save a single image slice

In [ ]:
# remove white border code: https://stackoverflow.com/questions/11837979/removing-white-space-around-a-saved-image
# uncomment savefig line to export image
# change slicenum to pick the slice you wish to save

slicenum = 13
dpi_ = 300
rows = 1
cols = 1
width = 10
height = (width*(rows/cols))

fig, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(width, height))
plt.gca().set_axis_off()
plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
            hspace = 0, wspace = 0)
plt.margins(0,0)

ax.imshow(ic_grid_dict['im_' + str(slicenum)])
plt.xticks([])
plt.yticks([])

plt.savefig(os.path.join(single_images_dir, 'cid_'+str(cellid)+'_mid_'+str(mitoid)+'_'+str(dpi)+'dpi'+'_slice_'+str(slicenum)+'.png'), dpi=dpi_)
plt.show()

In [ ]:
print(os.path.join(single_images_dir, 'cid_'+str(cellid)+'_mid_'+str(mitoid)+'_'+str(dpi)+'dpi'+'_slice_'+str(slicenum)+'.png'))

## Save the entire z-series as individual images to a unique subfolder
**Note:** these images will color the cellid of interest segementation

In [ ]:
# uncomment savefig line to save the entire zseries as individual images

dpi_ = 300
rows = 1
cols = 1
width = 10
height = (width*(rows/cols))

for i in range(zslices):
    fig, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(width, height))
    plt.gca().set_axis_off()
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0)
    plt.margins(0,0)
    ax.imshow(ic_grid_dict['im_' + str(i)])
    plt.xticks([])
    plt.yticks([])
    plt.savefig(os.path.join(imagestacks_dir, 'cid_'+str(cellid)+'_mid_'+str(mitoid)+'_'+str(dpi)+'dpi'+'_slice_'+str(i)+'.png'), dpi=dpi_)
    plt.close(fig)

In [ ]:
print(os.path.join(imagestacks_dir, 'cid_'+str(cellid)+'_mid_'+str(mitoid)+'_'+str(dpi)+'dpi'+'_slice_'+str(0)+'.png'))

In [ ]:
zslices

In [ ]:
type(ic_grid_dict['im_' + str(i)])

In [ ]:
type(fig)

# Color by mitochondria segmentation
All mitochondria within the image will be colored (note some mitochondria segmentations are fragmented or missing)  
The cellid of interest will not be colored in these images

In [ ]:
mito_cv = cloudvolume.CloudVolume(mito_source)
mito_cv.bounds

In [ ]:
mito_seg = mito_cv[(position_x_min)/2:(position_x_max)/2, (position_y_min)/2:(position_y_max)/2, position_z]

In [ ]:
# may be much faster than np.unique
uniq, cts = fastremap.unique(mito_seg, return_counts=True) 
# relabel values from 1 and refit data type
mito_seg, remapping = fastremap.renumber(mito_seg, in_place=True)

In [ ]:
remapping

In [ ]:
uniq

In [ ]:
tr_mito_seg = scipy.ndimage.rotate(mito_seg, 90)
plt.imshow(np.squeeze(tr_mito_seg), origin='lower')

In [ ]:
img_client = ImageryClient(image_source = img_source, segmentation_source=seg_source)
img_client_mito = ImageryClient(image_source = img_source, segmentation_source=mito_source)

In [ ]:
bounds = [
    [position_x_min, position_y_min, position_z],
    [position_x_max, position_y_max, position_z_max]
]

In [ ]:
print(bounds)

In [ ]:
image_size

In [ ]:
image = img_client.image_cutout(bounds)

In [ ]:
image.T

In [ ]:
Image.fromarray(image.T)

In [ ]:
# uncomment if you want to view the entire array
#import sys
#np.set_printoptions(threshold=sys.maxsize)

In [ ]:
#seg.T

In [ ]:
seg = img_client_mito.segmentation_cutout(bounds)

Image.fromarray( (seg.T / np.max(seg) * 255).astype('uint8') )

In [ ]:
uniq.tolist()[1:]

In [ ]:
uniq.tolist()[1:2]

In [ ]:
len(seg.T)

In [ ]:
# this code selects all mitochondria in the image to color, not just the mitoid of interest

root_ids = uniq.tolist()[1:]
#seg_ = img_client.segmentation_cutout(bounds, root_ids=root_ids)
Image.fromarray( (seg.T / np.max(seg) * 255).astype('uint8') )

In [ ]:
image, segs = img_client_mito.image_and_segmentation_cutout(bounds,
                                                       split_segmentations=True,
                                                       root_ids=root_ids)

In [ ]:
image

In [ ]:
# segs

In [ ]:
ic.composite_overlay(segs, imagery=image)

In [ ]:
# this is a repeat of the above code condensed into a single notebook cell

ctr = [position[0], position[1], position[2]+0] 
img_width = image_size
image = img_client.image_cutout(ctr, bbox_size=(img_width, img_width))
bounds = ic.bounds_from_center(ctr, width=img_width, height=img_width, depth=1)
image = img_client.image_cutout(bounds)
seg = img_client.segmentation_cutout(bounds)
root_ids = uniq.tolist()[1:]
seg = img_client_mito.segmentation_cutout(bounds, root_ids=root_ids)
split_seg = img_client_mito.split_segmentation_cutout(bounds, root_ids=root_ids)
image, segs = img_client_mito.image_and_segmentation_cutout(bounds, split_segmentations=True, root_ids=root_ids)
ic.composite_overlay(segs, imagery=image)

In [ ]:
# create a dictionary to store the mitochondria segmentation image of each slice
# this will be more time-consuming as the slices and/or image size increases

ic_mito_grid_dict = {}
for i in range(zslices):
    ctr = [position[0], position[1], position[2]+i] 
    img_width = image_size
    image = img_client.image_cutout(ctr, bbox_size=(img_width, img_width))
    bounds = ic.bounds_from_center(ctr, width=img_width, height=img_width, depth=1)
    image = img_client.image_cutout(bounds)
    seg = img_client_mito.segmentation_cutout(bounds)
    root_ids = uniq.tolist()[1:]
    seg = img_client_mito.segmentation_cutout(bounds, root_ids=root_ids)
    split_seg = img_client_mito.split_segmentation_cutout(bounds, root_ids=root_ids)
    image, segs = img_client_mito.image_and_segmentation_cutout(bounds, split_segmentations=True, root_ids=root_ids)
    ic_mito_grid_dict['im_mito_' + str(i)] = ic.composite_overlay(segs, imagery=image)
    
locals().update(ic_mito_grid_dict)

In [ ]:
# ic_mito_grid_dict

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(width, height))
plt.subplots_adjust(hspace=0)
plt.setp(ax, xticks=[], yticks=[])
ax.imshow(ic_mito_grid_dict['im_mito_' + str(0)])

# Save a montage image grid of all slices
Images will be colored by mitochondria segmentation

In [ ]:
# change desired DPI; 300 dpi will result in very large montage file sizes
# uncomment fig lines to generate and save the montage image

dpi = 100
rows = nrows
cols = ncols
width = 30
height = (width*((rows)/(cols)))

fig, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(width, height))
plt.subplots_adjust(hspace=0)
k=0
for i in range(rows):
    for j in range(cols):
        plt.setp(ax, xticks=[], yticks=[])
        ax[i,j].imshow(ic_mito_grid_dict['im_mito_' + str(k)])
        k=k+1

plt.savefig(os.path.join(imagemontages_dir, 'cid_'+str(cellid)+'_mid_'+str(mitoid)+'_zmontage_mitoseg_'+str(dpi)+'dpi.png'), dpi=dpi)
plt.show()

## Save a single image slice
Each mitochondria segmentation will be colored

In [ ]:
# remove white border code: https://stackoverflow.com/questions/11837979/removing-white-space-around-a-saved-image
# uncomment savefig line to export image
# change slicenum to pick the slice you wish to save

slicenum = 13
dpi_ = 300
rows = 1
cols = 1
width = 10
height = (width*(rows/cols))

fig, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(width, height))
plt.gca().set_axis_off()
plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
            hspace = 0, wspace = 0)
plt.margins(0,0)

ax.imshow(ic_mito_grid_dict['im_mito_' + str(slicenum)])
plt.xticks([])
plt.yticks([])

plt.savefig(os.path.join(single_images_dir, 'cid_'+str(cellid)+'_mid_'+str(mitoid)+'_'+str(dpi)+'dpi'+'_mitoseg_slice_'+str(slicenum)+'.png'), dpi=dpi_)
plt.show()

## Save the entire z-series as individual images to a unique subfolder
**Note:** these images will be colored by mitochondria segementation and not cellid 

In [ ]:
# uncomment savefig line to save the entire zseries as individual images

dpi_ = 300
rows = 1
cols = 1
width = 10
height = (width*(rows/cols))

for i in range(zslices):
    fig, ax = plt.subplots(nrows=rows, ncols=cols, figsize=(width, height))
    plt.gca().set_axis_off()
    plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0)
    plt.margins(0,0)
    ax.imshow(ic_mito_grid_dict['im_mito_' + str(i)])
    plt.xticks([])
    plt.yticks([])
    plt.savefig(os.path.join(imagestacks_dir, 'cid_'+str(cellid)+'_mid_'+str(mitoid)+'_'+str(dpi)+'dpi'+'_mitoseg_slice_'+str(i)+'.png'), dpi=dpi_)
    plt.close(fig)

## Re-run mitoid mapping using top position instead of bottom position

In [ ]:
top_position_x = top_position[0]
top_position_y = top_position[1]
top_position_z = top_position[2]

top_position_x_min = top_position_x - half_image_size
top_position_x_max = top_position_x + half_image_size
top_position_y_min = top_position_y - half_image_size
top_position_y_max = top_position_y + half_image_size
top_position_z_max = top_position_z + 1

In [ ]:
mito_seg = mito_cv[(top_position_x_min)/2:(top_position_x_max)/2, (top_position_y_min)/2:(top_position_y_max)/2, top_position_z]

In [ ]:
# may be much faster than np.unique
uniq, cts = fastremap.unique(mito_seg, return_counts=True) 
# relabel values from 1 and refit data type
mito_seg, remapping = fastremap.renumber(mito_seg, in_place=True)

In [ ]:
remapping

In [ ]:
uniq

## Save a single image slice using top position
Each mitochondria segmentation will be colored

In [ ]:
# this is a repeat of the above code condensed into a single notebook cell

ctr = [top_position[0], top_position[1], top_position[2]+0] 
img_width = image_size
image = img_client.image_cutout(ctr, bbox_size=(img_width, img_width))
bounds = ic.bounds_from_center(ctr, width=img_width, height=img_width, depth=1)
image = img_client.image_cutout(bounds)
seg = img_client.segmentation_cutout(bounds)
root_ids = uniq.tolist()[1:]
seg = img_client_mito.segmentation_cutout(bounds, root_ids=root_ids)
split_seg = img_client_mito.split_segmentation_cutout(bounds, root_ids=root_ids)
image, segs = img_client_mito.image_and_segmentation_cutout(bounds, split_segmentations=True, root_ids=root_ids)
ic.composite_overlay(segs, imagery=image)